In [26]:
!pip install pymed
!pip install mechanize
!pip install html2text
!pip install pandas

     |████████████████████████████████| 45 kB 863 kB/s eta 0:00:01
     |████████████████████████████████| 115 kB 2.0 MB/s eta 0:00:01


In [4]:
import pandas as pd
import requests
import io
import pymed
import mechanize
import re
import time
import sys
import html2text
import http.cookiejar
import numpy as np

In [27]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google'

# 1. Download all genes dataset from HGNC with OMIM ID

In [5]:
url = 'https://www.genenames.org/cgi-bin/download/custom?col=gd_hgnc_id&col=gd_app_sym&col=gd_app_name&col=gd_status&col=gd_prev_sym&col=gd_aliases&col=gd_pub_chrom_map&col=gd_pub_acc_ids&col=gd_pub_refseq_ids&col=md_mim_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit'

r = requests.get(url)
if r.ok:
    data = r.content.decode('utf8')
    df = pd.read_csv(io.StringIO(data), sep='\t')

df.shape

(47461, 10)

In [6]:
df[df['OMIM ID(supplied by OMIM)'].notnull()].shape

(16304, 10)

We hane 47461 genes and 16304 of them have OMIM ID

In [7]:
df.to_csv('gdrive/MyDrive/Биоинформатика/ИБ/Проект/Data/1_genenames.csv', index=False)
# df.to_csv('Data/1_genenames.csv', index=False)

# 2. Parse PubMed ID for genes which are in HGMD and OMIM

We create parcer which enter name of gene on http://www.hgmd.org/

If gene available it search OMIM link to gene and find all of the PubMed IDs for these gene

## 2.1. Login

In [9]:
hgmd_login_url = "http://www.hgmd.cf.ac.uk/docs/login.html"
email_address = "nokonstantinovskiy@edu.hse.ru"
password = "HGMD144550"

def initialize_browser():
    browser = mechanize.Browser()
    # Cookie Jar
    cj = http.cookiejar.LWPCookieJar()
    browser.set_cookiejar(cj)

    # Browser options
    browser.set_handle_equiv(True)
#    br.set_handle_gzip(True)
    browser.set_handle_redirect(True)
    browser.set_handle_referer(True)
    browser.set_handle_robots(False)

    # Follows refresh 0 but not hangs on refresh > 0
    browser.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)

    # Want debugging messages?
#    br.set_debug_http(True)
#    br.set_debug_redirects(True)
#    br.set_debug_responses(True)

    # User-Agent (this is cheating, ok?)
    browser.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]
    browser.addheaders.append(('email', email_address))

    return browser


def login_hgmd(browser):
    H2T = html2text.HTML2Text()

    response = browser.open(hgmd_login_url)
    html = response.read()

    # print response to STDOUT for debugging purposes
    # the html2text library is used for formatting the output in a more readable form
    print('Response:')
    print(html)
    print('---------')

    # print all the forms in the current page
    print('Forms:')
    print([f for f in browser.forms()])
    print('---------')

    # select login form
    browser.select_form(nr=0)
    print('Login form:')
    print(browser.form)
    print('---------')

    # print all controls in the current form, for debugging purposes
    print('Form controls:')
    print([c.name for c in browser.form.controls])
    print('---------')

    # set username and password
    browser.form['email'] = email_address
    browser.form['password'] = password

    # submit form
    response_form = browser.submit()

    # Now, you should have successfully logged in. The contents of the page will be changed. Check the contents of br.read()
    html_response = response_form.read()
    print('Login form response:')
    print(html_response)
    print('---------')

    # wait 2 seconds to not overload the server
    time.sleep(2)

    return browser

br = initialize_browser()
br = login_hgmd(br)

Response:
b'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">\n<html>\n<head>\n<link rel="shortcut icon" href="bar/hgmd_new.ico">\n<meta http-equiv="content-type" content="text/html; charset=UTF-8">\n<title>HGMD&reg; login</title>\n<link rel="stylesheet" type="text/css" href="../docs/login.css">\n</head>\n<body bgcolor="#FFFFFF">\n\n<table border=0 width="95%" cellpadding="1" align=center><tr> \n    <td align=left valign=middle width="15%"><a href="http://www.hgmd.cf.ac.uk/"><img src="../ac/bar/hgmd_logo.gif" alt="HGMD, Cardiff" border="0"></a></td>\n    <td align=center valign=middle><h3 align=center>HGMD&reg; Login</h3></td>\n    <td align=right valign=middle width="15%"><a href="http://www.biobase-international.com"><img src="../ac/bar/biobase_source.gif" alt="Biobase" border="0"></a></td>\n</table>\n<hr width="95%">\n\n<table border="0" cellpadding="1" width="95%" align=center>\n<tr><td>\n\n<table class="login" border=0 width="100%" cellpadding="3" align=center>\n<tr>

## 2.2. Parce

We will iterate over all genes with OMIM ID and will find all PubMed articles

In [10]:
df_omim = df[df['OMIM ID(supplied by OMIM)'].notnull()]
df_omim.head()

,HGNC ID,Approved symbol,Approved name,Status,Previous symbols,Alias symbols,Chromosome,Accession numbers,RefSeq IDs,OMIM ID(supplied by OMIM)
0,HGNC:5,A1BG,alpha-1-B glycoprotein,Approved,NaN,NaN,19q13.43,NaN,NM_130786,138670
2,HGNC:24086,A1CF,APOBEC1 complementation factor,Approved,NaN,"ACF, ASP, ACF64, ACF65, APOBEC1CF",10q11.23,AF271790,NM_014576,618199
4,HGNC:7,A2M,alpha-2-macroglobulin,Approved,NaN,"FWP007, S863-7, CPAMD5",12p13.31,"BX647329, X68728, M11313",NM_000014,103950
6,HGNC:23336,A2ML1,alpha-2-macroglobulin like 1,Approved,CPAMD9,"FLJ25179, p170",12p13.31,AK057908,NM_144670,610627
13,HGNC:18149,A4GALT,"alpha 1,4-galactosyltransferase (P blood group)",Approved,P1,"A14GALT, Gb3S, P(k)",22q13.2,NaN,NM_017436,607922


In [22]:
search_ngmd_url = 'http://www.hgmd.cf.ac.uk/ac/index.php'
gene_url = 'http://www.hgmd.cf.ac.uk/ac/gene.php?gene={}'

cur = 0
total = df_omim.shape[0]

genes_in_hgmd = 0
pubmed_articles_num = 0

# omim link pattern
pattern = re.compile('(http:\/\/omim\.org\/entry\/[0-9]*)')

# PubMed id pattern
pubmed_p = re.compile('\[PubMed\: ([0-9]*)\]')

# flag = False

result_pubmed_dict = {'gene_symb': [], 'omim_id': [], 'pubmed_id': []}

for (i, row) in df_omim.iterrows():
    print('Processed:', cur, '/', total, 'Genes in HGMD:', genes_in_hgmd, 'PubMed Articles Num:', pubmed_articles_num, )

    symb = row['Approved symbol']

    response = br.open(gene_url.format(symb))
    html = response.read().decode('utf-8')

    omim_links = pattern.findall(html)

    if len(omim_links) > 0:
        genes_in_hgmd += 1
#         flag = True
        omim_link = omim_links[0]
        
        response = br.open(omim_link)
        html = response.read().decode('utf-8')

        pubmed_ids = pubmed_p.findall(html)

        pubmed_articles_num += len(pubmed_ids)
        
        for pid in pubmed_ids:
            result_pubmed_dict['gene_symb'].append(symb)
            result_pubmed_dict['omim_id'].append(int(omim_link.split('/')[-1]))
            result_pubmed_dict['pubmed_id'].append(int(pid))
    
#     if flag:
#         break

    time.sleep(1)

    cur += 1

Processed: 0 / 16304 Genes in HGMD: 0 PubMed Articles Num: 0
Processed: 1 / 16304 Genes in HGMD: 0 PubMed Articles Num: 0
Processed: 2 / 16304 Genes in HGMD: 0 PubMed Articles Num: 0
Processed: 3 / 16304 Genes in HGMD: 1 PubMed Articles Num: 28
Processed: 4 / 16304 Genes in HGMD: 2 PubMed Articles Num: 32
Processed: 5 / 16304 Genes in HGMD: 3 PubMed Articles Num: 49
Processed: 6 / 16304 Genes in HGMD: 3 PubMed Articles Num: 49
Processed: 7 / 16304 Genes in HGMD: 4 PubMed Articles Num: 62
Processed: 8 / 16304 Genes in HGMD: 4 PubMed Articles Num: 62
Processed: 9 / 16304 Genes in HGMD: 5 PubMed Articles Num: 68
Processed: 10 / 16304 Genes in HGMD: 5 PubMed Articles Num: 68
Processed: 11 / 16304 Genes in HGMD: 6 PubMed Articles Num: 72
Processed: 12 / 16304 Genes in HGMD: 6 PubMed Articles Num: 72
Processed: 13 / 16304 Genes in HGMD: 6 PubMed Articles Num: 72
Processed: 14 / 16304 Genes in HGMD: 7 PubMed Articles Num: 86
Processed: 15 / 16304 Genes in HGMD: 7 PubMed Articles Num: 86
Proce

Processed: 127 / 16304 Genes in HGMD: 71 PubMed Articles Num: 1030
Processed: 128 / 16304 Genes in HGMD: 72 PubMed Articles Num: 1033
Processed: 129 / 16304 Genes in HGMD: 73 PubMed Articles Num: 1053
Processed: 130 / 16304 Genes in HGMD: 73 PubMed Articles Num: 1053
Processed: 131 / 16304 Genes in HGMD: 74 PubMed Articles Num: 1093
Processed: 132 / 16304 Genes in HGMD: 75 PubMed Articles Num: 1100
Processed: 133 / 16304 Genes in HGMD: 75 PubMed Articles Num: 1100
Processed: 134 / 16304 Genes in HGMD: 75 PubMed Articles Num: 1100
Processed: 135 / 16304 Genes in HGMD: 76 PubMed Articles Num: 1110
Processed: 136 / 16304 Genes in HGMD: 77 PubMed Articles Num: 1112
Processed: 137 / 16304 Genes in HGMD: 78 PubMed Articles Num: 1130
Processed: 138 / 16304 Genes in HGMD: 79 PubMed Articles Num: 1147
Processed: 139 / 16304 Genes in HGMD: 79 PubMed Articles Num: 1147
Processed: 140 / 16304 Genes in HGMD: 79 PubMed Articles Num: 1147
Processed: 141 / 16304 Genes in HGMD: 79 PubMed Articles Num: 

Processed: 249 / 16304 Genes in HGMD: 128 PubMed Articles Num: 1704
Processed: 250 / 16304 Genes in HGMD: 129 PubMed Articles Num: 1706
Processed: 251 / 16304 Genes in HGMD: 130 PubMed Articles Num: 1710
Processed: 252 / 16304 Genes in HGMD: 131 PubMed Articles Num: 1715
Processed: 253 / 16304 Genes in HGMD: 132 PubMed Articles Num: 1716
Processed: 254 / 16304 Genes in HGMD: 132 PubMed Articles Num: 1716
Processed: 255 / 16304 Genes in HGMD: 132 PubMed Articles Num: 1716
Processed: 256 / 16304 Genes in HGMD: 133 PubMed Articles Num: 1723
Processed: 257 / 16304 Genes in HGMD: 134 PubMed Articles Num: 1725
Processed: 258 / 16304 Genes in HGMD: 135 PubMed Articles Num: 1735
Processed: 259 / 16304 Genes in HGMD: 135 PubMed Articles Num: 1735
Processed: 260 / 16304 Genes in HGMD: 135 PubMed Articles Num: 1735
Processed: 261 / 16304 Genes in HGMD: 136 PubMed Articles Num: 1758
Processed: 262 / 16304 Genes in HGMD: 137 PubMed Articles Num: 1769
Processed: 263 / 16304 Genes in HGMD: 137 PubMed

Processed: 370 / 16304 Genes in HGMD: 192 PubMed Articles Num: 2387
Processed: 371 / 16304 Genes in HGMD: 192 PubMed Articles Num: 2387
Processed: 372 / 16304 Genes in HGMD: 193 PubMed Articles Num: 2389
Processed: 373 / 16304 Genes in HGMD: 194 PubMed Articles Num: 2392
Processed: 374 / 16304 Genes in HGMD: 195 PubMed Articles Num: 2400
Processed: 375 / 16304 Genes in HGMD: 195 PubMed Articles Num: 2400
Processed: 376 / 16304 Genes in HGMD: 195 PubMed Articles Num: 2400
Processed: 377 / 16304 Genes in HGMD: 196 PubMed Articles Num: 2403
Processed: 378 / 16304 Genes in HGMD: 197 PubMed Articles Num: 2411
Processed: 379 / 16304 Genes in HGMD: 198 PubMed Articles Num: 2430
Processed: 380 / 16304 Genes in HGMD: 198 PubMed Articles Num: 2430
Processed: 381 / 16304 Genes in HGMD: 199 PubMed Articles Num: 2431
Processed: 382 / 16304 Genes in HGMD: 200 PubMed Articles Num: 2440
Processed: 383 / 16304 Genes in HGMD: 201 PubMed Articles Num: 2474
Processed: 384 / 16304 Genes in HGMD: 201 PubMed

Processed: 491 / 16304 Genes in HGMD: 261 PubMed Articles Num: 3405
Processed: 492 / 16304 Genes in HGMD: 261 PubMed Articles Num: 3405
Processed: 493 / 16304 Genes in HGMD: 261 PubMed Articles Num: 3405
Processed: 494 / 16304 Genes in HGMD: 262 PubMed Articles Num: 3409
Processed: 495 / 16304 Genes in HGMD: 263 PubMed Articles Num: 3417
Processed: 496 / 16304 Genes in HGMD: 264 PubMed Articles Num: 3425
Processed: 497 / 16304 Genes in HGMD: 265 PubMed Articles Num: 3433
Processed: 498 / 16304 Genes in HGMD: 265 PubMed Articles Num: 3433
Processed: 499 / 16304 Genes in HGMD: 265 PubMed Articles Num: 3433
Processed: 500 / 16304 Genes in HGMD: 266 PubMed Articles Num: 3434
Processed: 501 / 16304 Genes in HGMD: 267 PubMed Articles Num: 3449
Processed: 502 / 16304 Genes in HGMD: 268 PubMed Articles Num: 3461
Processed: 503 / 16304 Genes in HGMD: 269 PubMed Articles Num: 3493
Processed: 504 / 16304 Genes in HGMD: 269 PubMed Articles Num: 3493
Processed: 505 / 16304 Genes in HGMD: 270 PubMed

...

Processed: 6219 / 16304 Genes in HGMD: 2909 PubMed Articles Num: 40038
Processed: 6220 / 16304 Genes in HGMD: 2909 PubMed Articles Num: 40038
Processed: 6221 / 16304 Genes in HGMD: 2910 PubMed Articles Num: 40062
Processed: 6222 / 16304 Genes in HGMD: 2910 PubMed Articles Num: 40062
Processed: 6223 / 16304 Genes in HGMD: 2910 PubMed Articles Num: 40062
Processed: 6224 / 16304 Genes in HGMD: 2911 PubMed Articles Num: 40063
Processed: 6225 / 16304 Genes in HGMD: 2911 PubMed Articles Num: 40063
Processed: 6226 / 16304 Genes in HGMD: 2911 PubMed Articles Num: 40063
Processed: 6227 / 16304 Genes in HGMD: 2911 PubMed Articles Num: 40063
Processed: 6228 / 16304 Genes in HGMD: 2912 PubMed Articles Num: 40067
Processed: 6229 / 16304 Genes in HGMD: 2912 PubMed Articles Num: 40067
Processed: 6230 / 16304 Genes in HGMD: 2913 PubMed Articles Num: 40072
Processed: 6231 / 16304 Genes in HGMD: 2914 PubMed Articles Num: 40073
Processed: 6232 / 16304 Genes in HGMD: 2914 PubMed Articles Num: 40073
Proces

Processed: 6335 / 16304 Genes in HGMD: 2971 PubMed Articles Num: 40773
Processed: 6336 / 16304 Genes in HGMD: 2971 PubMed Articles Num: 40773
Processed: 6337 / 16304 Genes in HGMD: 2971 PubMed Articles Num: 40773
Processed: 6338 / 16304 Genes in HGMD: 2971 PubMed Articles Num: 40773
Processed: 6339 / 16304 Genes in HGMD: 2971 PubMed Articles Num: 40773
Processed: 6340 / 16304 Genes in HGMD: 2972 PubMed Articles Num: 40793
Processed: 6341 / 16304 Genes in HGMD: 2972 PubMed Articles Num: 40793
Processed: 6342 / 16304 Genes in HGMD: 2973 PubMed Articles Num: 40823
Processed: 6343 / 16304 Genes in HGMD: 2974 PubMed Articles Num: 40863
Processed: 6344 / 16304 Genes in HGMD: 2974 PubMed Articles Num: 40863
Processed: 6345 / 16304 Genes in HGMD: 2975 PubMed Articles Num: 40868
Processed: 6346 / 16304 Genes in HGMD: 2975 PubMed Articles Num: 40868
Processed: 6347 / 16304 Genes in HGMD: 2975 PubMed Articles Num: 40868
Processed: 6348 / 16304 Genes in HGMD: 2975 PubMed Articles Num: 40868
Proces

Processed: 6451 / 16304 Genes in HGMD: 3019 PubMed Articles Num: 41522
Processed: 6452 / 16304 Genes in HGMD: 3020 PubMed Articles Num: 41530
Processed: 6453 / 16304 Genes in HGMD: 3020 PubMed Articles Num: 41530
Processed: 6454 / 16304 Genes in HGMD: 3021 PubMed Articles Num: 41539
Processed: 6455 / 16304 Genes in HGMD: 3022 PubMed Articles Num: 41541
Processed: 6456 / 16304 Genes in HGMD: 3022 PubMed Articles Num: 41541
Processed: 6457 / 16304 Genes in HGMD: 3022 PubMed Articles Num: 41541
Processed: 6458 / 16304 Genes in HGMD: 3023 PubMed Articles Num: 41543
Processed: 6459 / 16304 Genes in HGMD: 3023 PubMed Articles Num: 41543
Processed: 6460 / 16304 Genes in HGMD: 3023 PubMed Articles Num: 41543
Processed: 6461 / 16304 Genes in HGMD: 3024 PubMed Articles Num: 41544
Processed: 6462 / 16304 Genes in HGMD: 3024 PubMed Articles Num: 41544
Processed: 6463 / 16304 Genes in HGMD: 3025 PubMed Articles Num: 41582
Processed: 6464 / 16304 Genes in HGMD: 3025 PubMed Articles Num: 41582
Proces

Processed: 6567 / 16304 Genes in HGMD: 3092 PubMed Articles Num: 42793
Processed: 6568 / 16304 Genes in HGMD: 3093 PubMed Articles Num: 42795
Processed: 6569 / 16304 Genes in HGMD: 3094 PubMed Articles Num: 42801
Processed: 6570 / 16304 Genes in HGMD: 3094 PubMed Articles Num: 42801


KeyboardInterrupt: 

In [32]:
result_pubmed_df = pd.DataFrame(result_pubmed_dict)

result_pubmed_df.to_csv('gdrive/MyDrive/Биоинформатика/ИБ/Проект/Data/2_true_pubmedids_part1.csv', index=False)
# result_pubmed_df.to_csv('Data/2_true_pubmedids_part1.csv', index=False)

In [33]:
print('Number of genes:', len(result_pubmed_df.gene_symb.unique()))
print('Number of PubMed Articles:', len(result_pubmed_df.pubmed_id.unique()))

Number of genes: 3089
Number of PubMed Articles: 38451


# 3. Download titles and abstract from PubMed

## 3.1. Download random PubMed articles

In [46]:
# PubMed min and max id to download random samples
pubmed_min_id = 1
pubmed_max_id = 33297669

negative_class_size = 200000

# PubMed settings
tool_name = 'MyTool'
pubmed_query = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={}&tool={}&email={}'

# title pattern
title_pattern = re.compile('title[\s]*{[\s]*name[\s]*\"([^\}]+)')

# abstract pattern
abstract_pattern = re.compile('abstract[\s]*\"([^\"]+)')

def get_pubmed_title_and_abstract(id):
    title = None
    abstract = None

    r = requests.get(pubmed_query.format(id, tool_name, email_address))
    clean_text = r.text.replace('\n', '').replace('\t', '')
    titles = title_pattern.findall(clean_text)

    if len(titles) > 0:
        title = titles[0][:titles[0].rfind('"')]

        abstracts = abstract_pattern.findall(clean_text)
        if len(abstracts) > 0 and abstracts[0] is not None:
            abstract = abstracts[0]
    
    time.sleep(0.2)

    # print(id, title, abstract)

    return title, abstract

In [42]:
negative_ids = list(np.random.choice(pubmed_max_id - pubmed_min_id, negative_class_size, replace=False) + pubmed_min_id + 1)

results = { 'pubmed_id': [], 'title': [], 'abstract': [] }

cur = 0
real_num = 0

for i in range(len(negative_ids)):
    print('Processed:', cur, '/', negative_class_size, 'Not Nones:', real_num)

    id = negative_ids[i]
    res = get_pubmed_title_and_abstract(id)
    if res[0] is not None:
        results['pubmed_id'].append(id)
        results['title'].append(res[0])
        results['abstract'].append(res[1])
        real_num += 1

    cur += 1

NameError: name 'np' is not defined

In [ ]:
false_articles_df = pd.DataFrame(results)

false_articles_df.loc[false_articles_df.abstract.isna(), 'abstract'] = ''
false_articles_df.title = false_articles_df.title.str.split('",', n=1, expand=True)[0]
false_articles_df.title = false_articles_df.title.str.lstrip('[').str.rstrip('].')

false_articles_df.to_csv('gdrive/MyDrive/Биоинформатика/ИБ/Проект/Data/3_false_articles.csv', index=False)
# false_articles_df.to_csv('Data/3_false_articles.csv', index=False)

## 3.2. Download true PubMed articles

In [47]:
positive_ids = list(result_pubmed_df.pubmed_id.unique())

true_results = { 'pubmed_id': [], 'title': [], 'abstract': [] }

cur = 0
real_num = 0

for i in range(len(positive_ids)):
    print('Processed:', cur, '/', len(positive_ids), 'Not Nones:', real_num)

    id = positive_ids[i]
    res = get_pubmed_title_and_abstract(id)
    if res[0] is not None:
        true_results['pubmed_id'].append(id)
        true_results['title'].append(res[0])
        true_results['abstract'].append(res[1])
        real_num += 1

    cur += 1

Processed: 0 / 38451 Not Nones: 0
Processed: 1 / 38451 Not Nones: 1
Processed: 2 / 38451 Not Nones: 2
Processed: 3 / 38451 Not Nones: 3
Processed: 4 / 38451 Not Nones: 4
Processed: 5 / 38451 Not Nones: 5
Processed: 6 / 38451 Not Nones: 6
Processed: 7 / 38451 Not Nones: 7
Processed: 8 / 38451 Not Nones: 8
Processed: 9 / 38451 Not Nones: 9
Processed: 10 / 38451 Not Nones: 10
Processed: 11 / 38451 Not Nones: 11
Processed: 12 / 38451 Not Nones: 12
Processed: 13 / 38451 Not Nones: 13
Processed: 14 / 38451 Not Nones: 14
Processed: 15 / 38451 Not Nones: 15
Processed: 16 / 38451 Not Nones: 16
Processed: 17 / 38451 Not Nones: 17
Processed: 18 / 38451 Not Nones: 18
Processed: 19 / 38451 Not Nones: 19
Processed: 20 / 38451 Not Nones: 20
Processed: 21 / 38451 Not Nones: 21
Processed: 22 / 38451 Not Nones: 22
Processed: 23 / 38451 Not Nones: 23
Processed: 24 / 38451 Not Nones: 24
Processed: 25 / 38451 Not Nones: 25
Processed: 26 / 38451 Not Nones: 26
Processed: 27 / 38451 Not Nones: 27
Processed: 2

Processed: 222 / 38451 Not Nones: 222
Processed: 223 / 38451 Not Nones: 223
Processed: 224 / 38451 Not Nones: 224
Processed: 225 / 38451 Not Nones: 225
Processed: 226 / 38451 Not Nones: 226
Processed: 227 / 38451 Not Nones: 227
Processed: 228 / 38451 Not Nones: 228
Processed: 229 / 38451 Not Nones: 229
Processed: 230 / 38451 Not Nones: 230
Processed: 231 / 38451 Not Nones: 231
Processed: 232 / 38451 Not Nones: 232
Processed: 233 / 38451 Not Nones: 233
Processed: 234 / 38451 Not Nones: 234
Processed: 235 / 38451 Not Nones: 235
Processed: 236 / 38451 Not Nones: 236
Processed: 237 / 38451 Not Nones: 237
Processed: 238 / 38451 Not Nones: 238
Processed: 239 / 38451 Not Nones: 239
Processed: 240 / 38451 Not Nones: 240
Processed: 241 / 38451 Not Nones: 241
Processed: 242 / 38451 Not Nones: 242
Processed: 243 / 38451 Not Nones: 243
Processed: 244 / 38451 Not Nones: 244
Processed: 245 / 38451 Not Nones: 245
Processed: 246 / 38451 Not Nones: 246
Processed: 247 / 38451 Not Nones: 247
Processed: 2

Processed: 438 / 38451 Not Nones: 438
Processed: 439 / 38451 Not Nones: 439
Processed: 440 / 38451 Not Nones: 440
Processed: 441 / 38451 Not Nones: 441
Processed: 442 / 38451 Not Nones: 442
Processed: 443 / 38451 Not Nones: 443
Processed: 444 / 38451 Not Nones: 444
Processed: 445 / 38451 Not Nones: 445
Processed: 446 / 38451 Not Nones: 446
Processed: 447 / 38451 Not Nones: 447
Processed: 448 / 38451 Not Nones: 448
Processed: 449 / 38451 Not Nones: 449
Processed: 450 / 38451 Not Nones: 450
Processed: 451 / 38451 Not Nones: 451
Processed: 452 / 38451 Not Nones: 452
Processed: 453 / 38451 Not Nones: 453
Processed: 454 / 38451 Not Nones: 454
Processed: 455 / 38451 Not Nones: 455
Processed: 456 / 38451 Not Nones: 456
Processed: 457 / 38451 Not Nones: 457
Processed: 458 / 38451 Not Nones: 458
Processed: 459 / 38451 Not Nones: 459
Processed: 460 / 38451 Not Nones: 460
Processed: 461 / 38451 Not Nones: 461
Processed: 462 / 38451 Not Nones: 462
Processed: 463 / 38451 Not Nones: 463
Processed: 4

Processed: 654 / 38451 Not Nones: 654
Processed: 655 / 38451 Not Nones: 655
Processed: 656 / 38451 Not Nones: 656
Processed: 657 / 38451 Not Nones: 657
Processed: 658 / 38451 Not Nones: 658
Processed: 659 / 38451 Not Nones: 659
Processed: 660 / 38451 Not Nones: 660
Processed: 661 / 38451 Not Nones: 661
Processed: 662 / 38451 Not Nones: 662
Processed: 663 / 38451 Not Nones: 663
Processed: 664 / 38451 Not Nones: 664
Processed: 665 / 38451 Not Nones: 665
Processed: 666 / 38451 Not Nones: 666
Processed: 667 / 38451 Not Nones: 667
Processed: 668 / 38451 Not Nones: 668
Processed: 669 / 38451 Not Nones: 669
Processed: 670 / 38451 Not Nones: 670
Processed: 671 / 38451 Not Nones: 671
Processed: 672 / 38451 Not Nones: 672
Processed: 673 / 38451 Not Nones: 673
Processed: 674 / 38451 Not Nones: 674
Processed: 675 / 38451 Not Nones: 675
Processed: 676 / 38451 Not Nones: 676
Processed: 677 / 38451 Not Nones: 677
Processed: 678 / 38451 Not Nones: 678
Processed: 679 / 38451 Not Nones: 679
Processed: 6

KeyboardInterrupt: 

In [54]:
true_articles_df = pd.DataFrame(true_results)

true_articles_df.loc[true_articles_df.abstract.isna(), 'abstract'] = ''
true_articles_df.title = true_articles_df.title.str.split('",', n=1, expand=True)[0]
true_articles_df.title = true_articles_df.title.str.lstrip('[').str.rstrip('].')

true_articles_df.to_csv('gdrive/MyDrive/Биоинформатика/ИБ/Проект/Data/3_true_articles.csv', index=False)
# true_articles_df.to_csv('Data/3_true_articles.csv', index=False)

## 3.3. Remove articles from false df if it in true df

In [73]:
true_articles_df = pd.read_csv('gdrive/MyDrive/Биоинформатика/ИБ/Проект/Data/3_true_articles.csv').fillna('')
false_articles_df = pd.read_csv('gdrive/MyDrive/Биоинформатика/ИБ/Проект/Data/3_false_articles.csv').fillna('')
# true_articles_df = pd.read_csv('Data/3_true_articles.csv').fillna('')
# false_articles_df = pd.read_csv('Data/3_false_articles.csv').fillna('')

false_articles_df = false_articles_df[~false_articles_df.pubmed_id.isin(positive_ids)]

In [74]:
print('true_articles_df:', true_articles_df.shape[0])
print('false_articles_df:', false_articles_df.shape[0])

true_articles_df: 13923
false_articles_df: 19690


## 3.4. Concat DataSets

In [101]:
true_articles_df['label'] = int(True)
false_articles_df['label'] = int(False)

all_articles_df = pd.concat([true_articles_df, false_articles_df]).reset_index(drop=True)

In [102]:
# all_articles_df.to_csv('gdrive/MyDrive/Биоинформатика/ИБ/Проект/Data/3_all_articles.csv', index=False)
all_articles_df.to_csv('Data/3_all_articles.csv', index=False)

## 3.5. Create validation set

In [103]:
all_articles_df.label.sum() / all_articles_df.shape[0]

0.41421473834528305

In [104]:
test = all_articles_df.sample(2000)
test.label.sum() / test.shape[0]

0.413

In [105]:
all_articles_df.shape

(33613, 4)

In [107]:
TRAIN_articles_df = all_articles_df.drop(test.index)

In [108]:
TRAIN_articles_df.to_csv('gdrive/MyDrive/Биоинформатика/ИБ/Проект/Data/3_TRAIN_articles.csv', index=False)
test.to_csv('gdrive/MyDrive/Биоинформатика/ИБ/Проект/Data/3_TEST_articles.csv', index=False)
# TRAIN_articles_df.to_csv('Data/3_TRAIN_articles.csv', index=False)
# test.to_csv('Data/3_TEST_articles.csv', index=False)